In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import random
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with gzip.open('train_cases.pkl.gz','rb') as fp:
    train_cases = pickle.load(fp)
type(train_cases), len(train_cases)

(tuple, 2)

In [3]:
train_X = train_cases[0]
train_y = train_cases[1]
ntrain = train_X.shape[0]
type(train_X), train_X.shape, train_X.dtype, type(train_y), train_y.shape, train_y.dtype

(numpy.ndarray,
 (16384, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (16384, 1),
 dtype('float64'))

In [4]:
with gzip.open('valid_cases.pkl.gz','rb') as fp:
    valid_cases = pickle.load(fp)
valid_X = valid_cases[0]
valid_y = valid_cases[1]
nvalid = valid_X.shape[0]
type(valid_X), valid_X.shape, valid_X.dtype, type(valid_y), valid_y.shape, valid_y.dtype

(numpy.ndarray,
 (4096, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (4096, 1),
 dtype('float64'))

In [5]:
def choose_training_case():
    i = random.randint(0, ntrain-1)
    return(train_X[i,:,:,:], train_y[i,:])

def choose_validation_case():
    i = random.randint(0, nvalid-1)
    return(valid_X[i,:,:,:], valid_y[i,:])

In [746]:
ADD_CONST = 1e-5
CENTER = -2.5
SCALE = 5
XFORM = True

In [747]:
def xform(a):
    if XFORM:
        return((np.log(a+ADD_CONST) - CENTER) / SCALE)
    else:
        return(a)

In [748]:
def make_batch_from_megabatch(batchsize=32, validation=False):
    get_case = choose_validation_case if validation else choose_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        case = get_case()
        x = np.expand_dims(case[0], axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(case[1])).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(xform(xbatch), ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=True))

In [749]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [750]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [996]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, input_shape=input_shape)) # 64
#model.add(keras.layers.Conv2D(50, 1, input_shape=input_shape))
#model.add(keras.layers.ELU(1.0))
model.add(keras.layers.LeakyReLU(0))
#model.add(keras.layers.MaxPooling2D(2,1))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.SpatialDropout2D(.1))
model.add(keras.layers.Conv2D(32, 1))  # 32
#model.add(keras.layers.Conv2D(32, (2,1)))  # 32
#model.add(keras.layers.Conv2D(25, 1))  # 32
#model.add(keras.layers.PReLU())
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.LeakyReLU(0))
model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.SpatialDropout2D(.15))
#model.add(keras.layers.Conv2D(16, 1, activation='relu'))
#model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Conv2D(12, (1,3))) # 16
model.add(keras.layers.Conv2D(12, (1,4))) # 16
#model.add(keras.layers.Conv2D(14, (1,4))) # 16
#model.add(keras.layers.PReLU())
model.add(keras.layers.LeakyReLU(0.0))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.SpatialDropout2D(.3))
#model.add(keras.layers.SpatialDropout2D(.4))
#model.add(keras.layers.SpatialDropout2D(.2))
#model.add(keras.layers.Conv2D(25, (5,6))) # 29,5
model.add(keras.layers.Conv2D(25, (4,6))) # 29,5
#model.add(keras.layers.Conv2D(25, (4,7))) # 29,5
#model.add(keras.layers.Conv2D(25, 5)) # 29,5
#model.add(keras.layers.ELU(0.2))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.LeakyReLU(0.0))
model.add(keras.layers.BatchNormalization())

#model.add(keras.layers.Conv2D(10, (2,3)))
#model.add(keras.layers.LeakyReLU(0.0))
#model.add(keras.layers.SpatialDropout2D(.2))

#model.add(keras.layers.Conv2D(3, (2,5))) # 4,3
#model.add(keras.layers.Conv2D(4, (2,4))) # 4,3
#model.add(keras.layers.Conv2D(3, (1,5))) # 4,3
model.add(keras.layers.Conv2D(3, (1,4))) # 4,3
#model.add(keras.layers.Conv2D(4, 3)) # 4,3
model.add(keras.layers.LeakyReLU(0.0))
#model.add(keras.layers.MaxPooling2D(1,2))
#model.add(keras.layers.BatchNormalization())
#model.add(keras.layers.Conv2D(3, 1))
#model.add(keras.layers.LeakyReLU(0))
model.add(keras.layers.Flatten())
#model.add(keras.layers.Dropout(.9)) # or .2
model.add(keras.layers.Dropout(.8)) # or .2
#model.add(keras.layers.Dropout(.85)) # or .2
#model.add(keras.layers.GaussianDropout(.7)) # or .2
#model.add(keras.layers.Dense(25, activation='relu'))
#model.add(keras.layers.Dense(24))
#model.add(keras.layers.Dense(30))
model.add(keras.layers.Dense(20))
model.add(keras.layers.PReLU())
#model.add(keras.layers.LeakyReLU(0.0))
#model.add(keras.layers.Dropout(.2)) # .5 or .6
#model.add(keras.layers.Dropout(.15)) # .5 or .6
#model.add(keras.layers.GaussianDropout(.1))
model.add(keras.layers.Dense(1, activation='sigmoid'))
#                             kernel_regularizer=keras.regularizers.l1(2e-6)))

In [997]:
model.compile(loss='binary_crossentropy',
#              optimizer=keras.optimizers.Adam(lr=.003,decay=1e-4), 
              optimizer=keras.optimizers.Adam(lr=.003,decay=2e-4), 
              metrics=['accuracy'])

In [998]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_981 (Conv2D)          (None, 16, 16, 64)        8256      
_________________________________________________________________
leaky_re_lu_981 (LeakyReLU)  (None, 16, 16, 64)        0         
_________________________________________________________________
spatial_dropout2d_396 (Spati (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_982 (Conv2D)          (None, 16, 16, 32)        2080      
_________________________________________________________________
leaky_re_lu_982 (LeakyReLU)  (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_391 (Bat (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_983 (Conv2D)          (None, 16, 13, 12)        1548      
__________

In [999]:
batchsize = 32
epochs = 60
steps = 2**11/batchsize
valid=valid_gen(batchsize)
train=train_gen(batchsize)
model.fit_generator(train, 
                    validation_data=valid, validation_steps=steps,
                    epochs=60, 
                    steps_per_epoch=steps)

Epoch 1/60
64/64 [==============================] - 25s 394ms/step - loss: 0.6965 - acc: 0.7280 - val_loss: 0.5910 - val_acc: 0.7637
Epoch 2/60
64/64 [==============================] - 2s 32ms/step - loss: 0.6003 - acc: 0.7461 - val_loss: 0.5847 - val_acc: 0.7705
Epoch 3/60
64/64 [==============================] - 2s 32ms/step - loss: 0.5636 - acc: 0.7690 - val_loss: 0.5837 - val_acc: 0.7490
Epoch 4/60
64/64 [==============================] - 2s 32ms/step - loss: 0.5486 - acc: 0.7710 - val_loss: 0.5530 - val_acc: 0.7671
Epoch 5/60
64/64 [==============================] - 2s 31ms/step - loss: 0.5438 - acc: 0.7671 - val_loss: 0.5594 - val_acc: 0.7754
Epoch 6/60
64/64 [==============================] - 2s 31ms/step - loss: 0.5313 - acc: 0.7764 - val_loss: 0.5408 - val_acc: 0.7593
Epoch 7/60
64/64 [==============================] - 2s 32ms/step - loss: 0.5427 - acc: 0.7544 - val_loss: 0.5376 - val_acc: 0.7593
Epoch 8/60
64/64 [==============================] - 2s 32ms/step - loss: 0.5132 -

In [1000]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(xform(c), axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

BothSidesNow 41
Predicted:  0.0022004712    Actual:  False 

SmoothOperator 69
Predicted:  9.379174e-10    Actual:  False 

IThinkWereAloneNow2MP 44
Predicted:  6.0262482e-05    Actual:  False 

ItsRainingMen 185
Predicted:  0.00019341834    Actual:  False 

BloodyMerryMorning 74
Predicted:  0.005370181    Actual:  False 

ChatanoogaChooChoo 76
Predicted:  0.09146364    Actual:  False 

ImLookingThruYou 115
Predicted:  8.657952e-05    Actual:  False 

ImLookingThruYou 172
Predicted:  0.9228154    Actual:  True 

ChatanoogaChooChoo 57
Predicted:  0.005663717    Actual:  False 

Kalamazoo 40
Predicted:  0.0001505687    Actual:  False 



<code>
decay 0001      -> 1975, 2040, 2101, 2125, 2131, 2139, 2151, 2167, 2176, 2182, 
                   2195, 2195, 2198, 2202, *2204*, **2206**, **2223**, *2239*, 2270, 2287, 
                   2291, 2296, 2306, 2309, 2318, 2333, 2344, 2368, 2397, 2424, 2431, 2540
add conv layer  -> 2396, 2462
b/n before relu -> 5191, 5434
dense to 30     -> 2273, 2278, 2301
dense to 27     -> 2404
dense to 25     -> 2242
dense to 24     -> 2020, 2029, 2090,  
                   2100, 2133, 2140, 2154, 2175, 2195, 2198, 2211
                   2223, 2235, 2246, 2274, 2295, 2313, 2323, 2358, 2439, 2458
dense to 23     -> 2227, 2326
20 d/o middle   -> 2087, 2099, 2102, 2125, 2135, 2184, 2192, 
                   2195, 2200, *2206*, **2214**, **2221**, *2223*, 2235, 2252, 
                   2301, 2305, 2310, 2312, 2319, 2347, 2455
(32 2x1) below  -> 1987, 2198, *2294*, **2303**, *2337*, 2429, 2680
(3, 1x5)        -> 1967, 2018, 2019, 2048, 2053, 2058, 2071, 2073, 2076, 2077, 2081,
                   2089, 2089, 2094, 2101, 2103, 2107, 2118, 2119, 2120, 2128, 
                   2136, 2142, 2147, 2151, 2174, 2180, 2181, 2182, 
                   2188, 2197, *2200*, **2204**, *2214*, 2215, 2215, 2219
                   2226, 2232, 2234, 2235, 2243, 2243, 2244, 2253, 2261, 2264, 
                   2274, 2276, 2301, 2307, 2309, 2312, 2318, 2341, 2349, 2366, 
                   2389, 2397, 2407, 2467, 2478, 2520, 2618, 5494,
(3, 2x4)        -> 2155, 2161, *2187*, **2305**, *2384*, 2408, 2423
(12, 1x3)       -> 2181, 2230, 2267, 2343, 2373
(12, 1x5)       -> 2605, 245841 - val_acc: 0.9023
(12, 2x4)       -> 2222, 2843
(25, 5x6)       -> 2201, 2308, 2324
(25, 3x6)       -> 2511, 2438
(25, 4x7)       -> 2082, 2278, 2384, 2442
(25, 4x5)       -> 2273, 2303, 2344, 5460, 
    
d/o = .85       -> 2049, 2057, 2110, 2122, 2123, 2124, 2132, 2143, 2150, 2165, 2166, 2168, 
                   2171, 2192, 2193, 2197, *2216*, **2225** **2241**, *2245*, 2257, 2258, 2258,
                   2263, 2268, 2271, 2282, 2286, 2306, 2315, 2331, 2332, 2340, 2359, 2393, 5327
d/o = .9        -> 2406, 2526
d/o = .75       -> 2188, 2121, 2169, 
                   2192, *2227*, **2248** *2301*, 2327, 
                   2329, 2329, 2331,
40 d/o middle   -> 2087, 2298, 2412, 2436
leak .2 @32layr -> 2003, 2107, 2111, 2112, 2126, 2130, 2145, 2147, 
                   2156, 2161, 2169, 2183, 2196, 2209, *2220*, **2225**, *2239*, 2279, 2287,
                   2300, 2311, 2321, 2313, 2348, 2363, 2369, 2386, 2395, 2401, 2509, 2750

dense to 16     -> 2108, 2146, 2160, 
                   2194, 2212, *2212*, **2227**, *2253*, 2296, 2331,
                   2375, 2497, 2660
25 d/o middle ->   2267, 2291, 2294, 2324, 2399
32f to 25 below -> 2078, 2078, 2104, 2227, *2229*, **2236**, *2273*, 2297, 2298, 2405, 2428
32f to 50 below -> 2037, 2161, 2220, *2237*, **2245**, *2256*, 2280, 2339, 2471
(32 1x2) below  -> 2100, 2140, 2167, 2189, *2233*, **2267**, *2295*, 2317, 2339, 2411, 2426
decay 00015     -> 2061, 2185, 2234, **2251**, 2323, 2397, 2469, 2532?
    
baseline from below:
.8/20/prelu/0   -> 2040, 2034, 2052, 2054, 2056, 2058, 2074, 2075, 2078, 2091, 2096, 
                   2105, 2114, 2120, 2127, 2127, 2132, 2133, 2134, 2139, 2139, 2140. 2140, 2158, 
                   2162, 2164, 2166, 2166. 2168, 2172, 2174, 2178, 2182, 2188,
                   *2190*, **2190**, **2201**, *2203*, 2205, 2209, 2216, 2216, 
                   2217, 2221, 2233, 2241, 2242, 2242, 2247, 2256, 2269, 2287, 2292, 
                   2284, 2290, 2291, 2296, 2298, 2300, 2305, 2306, 2317, 2334, 2336, 
                   2349, 2351, 2355, 2389, 2428, 2480, 2483, 3374,
</code>

<code>
2nd to elu.2 -> 2191, *2230*, 2145, 2262, 2180, 2310, *2200*, 2250, 2168, **2224**, 1961,
                2268, 2247                
top to elu.3 -> 2370, 2568
relu0 to elu -> 2083, 2106, 2310, *2216*, 2105, 2334, 2092, **2288**, *2297*, 2420, 2221
gaussian 0.7 -> 2450
gaussian d/o -> 2439
(32 2x1)     -> **2303**, 1987, 2198, **2294**, 2429, 2680
(32 1x2)     -> 2100, 2140, 2167, *2189*, **2233**, *2267*, 2295, 2411, 2426
64f to 50    -> 2214, 2315, 2096, **2355**, 2387
32f to 50    -> 2161, 2237, 2256, **2245**, 2471
32f to 25    -> 2078, **2229**, 2104, *2273*, 2428, *2227*, 2078, 2298, 2405
32f to 20    -> 2265, 2081, *2201*, 2102, 2160, *2254*, 2301, **2236**, 2264
bottom 80    -> 2041, 2153, 2376, **2263**, *2261*, 2168, 2319, 2438, 2187, *2315*, 2331
batchsize=64 -> 2349, 2339
batchsize=16 -> 2360, 2315
steps/2      -> 2332, 2248 
stepsx2      -> 2233, **2205**, 2093, 2186, 2412, 2097, *2198*, 2097, *2217*, 2324, 2166,
                2354, 2259
    
baseline from below:
*.8/20/prelu/0   -> *2216*, 2172, 2317, 2247, 2300, 2074, 2091, 2158, 2040, 2296, 2120,
                   **2205**, *2190*, 2292, 2190, 2139, 2355, 2188, 2241, 2164, 2305,
                   2166, 2291, 2336, 2174, 2162, 2217, 2351, 2078, 2349, 2168,
                   2287, 3374
</code>

<code>
below w l2=1e-6 -> 2219, ?
maybe that or l1-> 2068, 2170, 2165, 2136, 2269, 2213, 2187, 2363, 2180, 2583,
                   2249, 2189, 2268, 2274, 2110, 2153
below w l2=5e-6 -> 2521
below w l2=2e-5 -> 2258, 2279 
    
*.8/20/prelu/0   -> 2216, 2172, 2317, 2247, 2300, 2074, 2091, 2158, 2040, 2296, 2120,
                   *2205*, **2190**, 2292, *2190*, 2139, 2355, 2188, 2241, 2164, 2305,
                   2166, 2291,  *
                   
above w l1=1e-5 -> 2232
above w l1=2e-6 -> *2222*, 2334, 2050, 2065, **2244**, 2325, 2136, 2091, 2364, 2201, 2281,
                   *2244*, 2384
 

try again, from below:
.7/20/prelu/0   -> *2303*, 2232, 2334, 2086, 2179, 2319, 2127, 2372, 2207, *2243*, **2270**,
                   2185, 2353, 2580, 2218, **2244**, 2341, 2331

.7/20/prelu/.15 -> 2141, 2321, 2381, 2268, 2386
.7/20/prelu/.2  -> 2181, 2140, 2285, 2173, *2257*, **2256**, 2212, *2243*, 2321, 2492, 2306
    
back to relu, from below:
dense .7/20/0   -> 2255, 2191, 2404, 2185, 2115, 2131, 2276, 2389, 2142, 2165, 2100,
                   2386, 2352, 2209, 2142, **2228**, 2470, 2267, 2423, 2218, 2487,
                   2290, 2323, *2223*, 2160, *2237*, 2433, 2103, 2221, 2287, 2244,
                   2364, 2098, 2100, 2120
                  
    
14f mid layer   -> 2213, 2308, 2504, 2351
10f mid layer   -> 2392, 2351
10f mid layr 2x4-> 2076, 2398, 2370, 2420
decay 0001      -> *2246*, 2288, 2087, 2096, 2342, 2155, 2149, 2057, 2163, **2246**, 2417,
                   *2273*, 2602  
decay 0003      -> 2403, 2356
decay 0004      -> 2156, **2278**, *2197*, 2356, 2372, 2140, *2333*
    
baseline from below:
.7/20/prelu/.1 -> 2173, 2176, 2267, 2138, 2216, 2408, 2121, 2424, 2166, 2282, 2230,
                   2274, 2227, 2362, 2157, 2274, 2147, 2189, 2413, 2398, 2193,
                   2276, 2317, 2331, 2117, 2234, 2217, **2247**, 2317, 2259, *2241*,
                   *2255*, 2366
                   
</code>

<code>
below g/d/o @.7 -> 2259, 2289, 2288, 2262
*.7/20/prelu/.1 -> 2173, 2176, 2267, 2138, *2216*, 2408, 2121, 2424, 2166, 2282, *2230*,
                   2274, **2227**, 2362, 2157, 2274, 2147 *
above g/d/o @.1 -> 2334, 2229, 2355, 
dense .7/20/.1  -> 2400, 2156, 2296, 2416
.7/20/prelu/0   -> 2303, 2232, 2334, 
dense .8/20/0   -> 2360, 2159, 2384, **2291**, 2288
dense .6/20/0   -> 2474, 2352
dense .7/25/0   -> 2363, 2266, 2284
dense .7/15/0   -> 2318, 2234, 2333, 2334
dense .7/20/0   -> 2255, 2191, 2404, 2185, 2115, 2131, 2276, 2389, 2142, 2165, 2100,
                  2386, 2352, 2209, 2142, **2228**, 2470, 2267, 2423, 2218, 2487,
                  2290, 2323, *2223*, 2160, *2237*, 2433, 2103, 2221
                  
dense .6/15/.1  -> 2044, 2213, *2268*, 2179, *2233*, 2334, 2422, **2256** 2334
dense .5/12/.3  -> 2163, 2271, 2353, 2325
dense .4/10/.6  -> 2731
    
baseline from before:
(12,1x4) -> 2282, *2250*, 2259, **2247**, 2233, 2155, 2174, 2493, 2228, 2386, 2060,
            2049, 2008, 2621, *2237*, 2359, 2212, 2354, 2273